In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 20
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.7e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6
for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t> start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, r, _, _, c = env.transition(state, ac, para)  
            s0 += r
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 25495860.0, E: 140.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-286.59
 Time: 0 | S: 25495860.0, E: 140.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-286.59
 Time: 0 | new_e:66.1, cum_e:66.1, new_i:59.1, cum_i:59.1
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495749.8, E: 184.8, A: 0.0, I: 59.1, D: 0.0, R: 6.4, Budget: 2700000.00, reward:-692.94
 Time: 1 | S: 25495746.0, E: 179.0, A: 0.0, I: 59.1, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-692.94
 Time: 1 | new_e:92.9, cum_e:159.0, new_i:81.7, cum_i:140.8
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 

 Time: 10 | S: 25376867.4, E: 455.1, A: 1536.6, I: 2334.3, D: 4.3, R: 114802.3, Budget: 2592556.99, reward:-15891.70
 Time: 10 | S: 25377021.0, E: 445.0, A: 1536.6, I: 2334.3, D: 4.3, R: 114786.0, Budget: 2592556.99, reward:-15891.70
 Time: 10 | new_e:527.4, cum_e:3588.7, new_i:565.0, cum_i:3455.1
[[0.00092405 0.00092405 0.00092405 0.00092405 0.00092405 0.00092405
  0.00092405 0.00092405 0.00092405 0.00092405 0.00092405 0.00092405
  0.00092405 0.00092405 0.00092405 0.00092405 0.00092405 0.00092405
  0.00092405 0.00092405]
 [0.51447389 0.51447389 0.51447389 0.51447389 0.51447389 0.51447389
  0.51447389 0.51447389 0.51447389 0.51447389 0.51447389 0.51447389
  0.51447389 0.51447389 0.51447389 0.51447389 0.51447389 0.51447389
  0.51447389 0.51447389]
 [0.60548399 0.60548399 0.60548399 0.60548399 0.60548399 0.60548399
  0.60548399 0.60548399 0.60548399 0.60548399 0.60548399 0.60548399
  0.60548399 0.60548399 0.60548399 0.60548399 0.60548399 0.60548399
  0.60548399 0.60548399]
 [0.87058477 0

 Time: 16 | S: 25198531.4, E: 283.5, A: 1698.5, I: 3483.3, D: 11.8, R: 291991.6, Budget: 2415980.69, reward:-23187.01
 Time: 16 | S: 25198744.0, E: 280.0, A: 1698.5, I: 3483.3, D: 11.8, R: 291994.0, Budget: 2415980.69, reward:-23187.01
 Time: 16 | new_e:146.7, cum_e:5174.5, new_i:310.1, cum_i:5899.0
[[0.23677221 0.23677221 0.23677221 0.23677221 0.23677221 0.23677221
  0.23677221 0.23677221 0.23677221 0.23677221 0.23677221 0.23677221
  0.23677221 0.23677221 0.23677221 0.23677221 0.23677221 0.23677221
  0.23677221 0.23677221]
 [0.84740506 0.84740506 0.84740506 0.84740506 0.84740506 0.84740506
  0.84740506 0.84740506 0.84740506 0.84740506 0.84740506 0.84740506
  0.84740506 0.84740506 0.84740506 0.84740506 0.84740506 0.84740506
  0.84740506 0.84740506]
 [0.98325199 0.98325199 0.98325199 0.98325199 0.98325199 0.98325199
  0.98325199 0.98325199 0.98325199 0.98325199 0.98325199 0.98325199
  0.98325199 0.98325199 0.98325199 0.98325199 0.98325199 0.98325199
  0.98325199 0.98325199]
 [0.99646241

 Time: 22 | S: 25006714.8, E: 235.1, A: 1313.9, I: 3610.5, D: 20.2, R: 484105.5, Budget: 2285901.73, reward:-29606.61
 Time: 22 | S: 25006972.0, E: 240.0, A: 1313.9, I: 3610.5, D: 20.2, R: 484107.0, Budget: 2285901.73, reward:-29606.61
 Time: 22 | new_e:293.3, cum_e:6544.4, new_i:361.5, cum_i:7928.7
[[0.01769949 0.01769949 0.01769949 0.01769949 0.01769949 0.01769949
  0.01769949 0.01769949 0.01769949 0.01769949 0.01769949 0.01769949
  0.01769949 0.01769949 0.01769949 0.01769949 0.01769949 0.01769949
  0.01769949 0.01769949]
 [0.96510364 0.96510364 0.96510364 0.96510364 0.96510364 0.96510364
  0.96510364 0.96510364 0.96510364 0.96510364 0.96510364 0.96510364
  0.96510364 0.96510364 0.96510364 0.96510364 0.96510364 0.96510364
  0.96510364 0.96510364]
 [0.85205728 0.85205728 0.85205728 0.85205728 0.85205728 0.85205728
  0.85205728 0.85205728 0.85205728 0.85205728 0.85205728 0.85205728
  0.85205728 0.85205728 0.85205728 0.85205728 0.85205728 0.85205728
  0.85205728 0.85205728]
 [0.71807345

 Time: 28 | S: 24822774.7, E: 151.6, A: 1190.6, I: 3162.0, D: 25.5, R: 668695.5, Budget: 2104173.79, reward:-34221.28
 Time: 28 | S: 24823072.0, E: 147.0, A: 1190.6, I: 3162.0, D: 25.5, R: 668687.0, Budget: 2104173.79, reward:-34221.28
 Time: 28 | new_e:74.9, cum_e:7529.6, new_i:193.7, cum_i:9553.7
[[0.02439182 0.02439182 0.02439182 0.02439182 0.02439182 0.02439182
  0.02439182 0.02439182 0.02439182 0.02439182 0.02439182 0.02439182
  0.02439182 0.02439182 0.02439182 0.02439182 0.02439182 0.02439182
  0.02439182 0.02439182]
 [0.79289805 0.79289805 0.79289805 0.79289805 0.79289805 0.79289805
  0.79289805 0.79289805 0.79289805 0.79289805 0.79289805 0.79289805
  0.79289805 0.79289805 0.79289805 0.79289805 0.79289805 0.79289805
  0.79289805 0.79289805]
 [0.99342019 0.99342019 0.99342019 0.99342019 0.99342019 0.99342019
  0.99342019 0.99342019 0.99342019 0.99342019 0.99342019 0.99342019
  0.99342019 0.99342019 0.99342019 0.99342019 0.99342019 0.99342019
  0.99342019 0.99342019]
 [0.99474767 

...

 Time: 64 | S: 24189278.2, E: 35.3, A: 420.5, I: 1283.5, D: 55.1, R: 1304927.3, Budget: 1319400.98, reward:-52165.82
 Time: 64 | S: 24189494.0, E: 38.0, A: 420.5, I: 1283.5, D: 55.1, R: 1304989.0, Budget: 1319400.98, reward:-52165.82
 Time: 64 | new_e:20.0, cum_e:11270.3, new_i:62.2, cum_i:15239.9
[[0.45258919 0.45258919 0.45258919 0.45258919 0.45258919 0.45258919
  0.45258919 0.45258919 0.45258919 0.45258919 0.45258919 0.45258919
  0.45258919 0.45258919 0.45258919 0.45258919 0.45258919 0.45258919
  0.45258919 0.45258919]
 [0.86597579 0.86597579 0.86597579 0.86597579 0.86597579 0.86597579
  0.86597579 0.86597579 0.86597579 0.86597579 0.86597579 0.86597579
  0.86597579 0.86597579 0.86597579 0.86597579 0.86597579 0.86597579
  0.86597579 0.86597579]
 [0.99613851 0.99613851 0.99613851 0.99613851 0.99613851 0.99613851
  0.99613851 0.99613851 0.99613851 0.99613851 0.99613851 0.99613851
  0.99613851 0.99613851 0.99613851 0.99613851 0.99613851 0.99613851
  0.99613851 0.99613851]
 [0.96983922 0

 Time: 70 | S: 24111545.2, E: 23.7, A: 207.2, I: 901.5, D: 57.1, R: 1383265.3, Budget: 1147709.67, reward:-52679.84
 Time: 70 | S: 24111740.0, E: 20.0, A: 207.2, I: 901.5, D: 57.1, R: 1383334.0, Budget: 1147709.67, reward:-52679.84
 Time: 70 | new_e:14.5, cum_e:11362.6, new_i:34.4, cum_i:15504.0
[[0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519
  0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519
  0.73052519 0.73052519 0.73052519 0.73052519 0.73052519 0.73052519
  0.73052519 0.73052519]
 [0.84333    0.84333    0.84333    0.84333    0.84333    0.84333
  0.84333    0.84333    0.84333    0.84333    0.84333    0.84333
  0.84333    0.84333    0.84333    0.84333    0.84333    0.84333
  0.84333    0.84333   ]
 [0.96387489 0.96387489 0.96387489 0.96387489 0.96387489 0.96387489
  0.96387489 0.96387489 0.96387489 0.96387489 0.96387489 0.96387489
  0.96387489 0.96387489 0.96387489 0.96387489 0.96387489 0.96387489
  0.96387489 0.96387489]
 [0.99305977 0.99305977 0

 Time: 76 | S: 23933769.4, E: 6.1, A: 107.1, I: 533.6, D: 58.0, R: 1561525.7, Budget: 940120.83, reward:-52870.27
 Time: 76 | S: 23933933.0, E: 19.0, A: 107.1, I: 533.6, D: 58.0, R: 1561587.0, Budget: 940120.83, reward:-52870.27
 Time: 76 | new_e:2.9, cum_e:11395.8, new_i:14.0, cum_i:15627.5
[[0.01001855 0.01001855 0.01001855 0.01001855 0.01001855 0.01001855
  0.01001855 0.01001855 0.01001855 0.01001855 0.01001855 0.01001855
  0.01001855 0.01001855 0.01001855 0.01001855 0.01001855 0.01001855
  0.01001855 0.01001855]
 [0.87340482 0.87340482 0.87340482 0.87340482 0.87340482 0.87340482
  0.87340482 0.87340482 0.87340482 0.87340482 0.87340482 0.87340482
  0.87340482 0.87340482 0.87340482 0.87340482 0.87340482 0.87340482
  0.87340482 0.87340482]
 [0.99944532 0.99944532 0.99944532 0.99944532 0.99944532 0.99944532
  0.99944532 0.99944532 0.99944532 0.99944532 0.99944532 0.99944532
  0.99944532 0.99944532 0.99944532 0.99944532 0.99944532 0.99944532
  0.99944532 0.99944532]
 [0.99775617 0.99775

 Time: 82 | S: 23856086.2, E: 2.1, A: 47.1, I: 298.8, D: 58.5, R: 1639507.3, Budget: 736291.41, reward:-52943.47
 Time: 82 | S: 23856238.0, E: 0.0, A: 47.1, I: 298.8, D: 58.5, R: 1639573.0, Budget: 736291.41, reward:-52943.47
 Time: 82 | new_e:1.1, cum_e:11405.5, new_i:6.0, cum_i:15680.2
[[0.87173016 0.87173016 0.87173016 0.87173016 0.87173016 0.87173016
  0.87173016 0.87173016 0.87173016 0.87173016 0.87173016 0.87173016
  0.87173016 0.87173016 0.87173016 0.87173016 0.87173016 0.87173016
  0.87173016 0.87173016]
 [0.65255403 0.65255403 0.65255403 0.65255403 0.65255403 0.65255403
  0.65255403 0.65255403 0.65255403 0.65255403 0.65255403 0.65255403
  0.65255403 0.65255403 0.65255403 0.65255403 0.65255403 0.65255403
  0.65255403 0.65255403]
 [0.99233625 0.99233625 0.99233625 0.99233625 0.99233625 0.99233625
  0.99233625 0.99233625 0.99233625 0.99233625 0.99233625 0.99233625
  0.99233625 0.99233625 0.99233625 0.99233625 0.99233625 0.99233625
  0.99233625 0.99233625]
 [0.99850857 0.99850857 

 Time: 88 | S: 23608960.2, E: 3.4, A: 23.5, I: 166.9, D: 58.8, R: 1886787.2, Budget: 592770.58, reward:-53009.83
 Time: 88 | S: 23609128.0, E: 0.0, A: 23.5, I: 166.9, D: 58.8, R: 1886855.0, Budget: 592770.58, reward:-53009.83
 Time: 88 | new_e:1.7, cum_e:11418.0, new_i:4.0, cum_i:15710.2
[[0.38243006 0.38243006 0.38243006 0.38243006 0.38243006 0.38243006
  0.38243006 0.38243006 0.38243006 0.38243006 0.38243006 0.38243006
  0.38243006 0.38243006 0.38243006 0.38243006 0.38243006 0.38243006
  0.38243006 0.38243006]
 [0.96053127 0.96053127 0.96053127 0.96053127 0.96053127 0.96053127
  0.96053127 0.96053127 0.96053127 0.96053127 0.96053127 0.96053127
  0.96053127 0.96053127 0.96053127 0.96053127 0.96053127 0.96053127
  0.96053127 0.96053127]
 [0.99666941 0.99666941 0.99666941 0.99666941 0.99666941 0.99666941
  0.99666941 0.99666941 0.99666941 0.99666941 0.99666941 0.99666941
  0.99666941 0.99666941 0.99666941 0.99666941 0.99666941 0.99666941
  0.99666941 0.99666941]
 [0.98594403 0.98594403 

In [6]:
(time.time() - tic) / ( T - start_time + 1)

87.35065793991089